In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [ ]:
# --- 2. ĐỌC VÀ KHÁM PHÁ DỮ LIỆU ---
# Tải file 'online_shoppers_purchasing_intention.csv' vào cùng thư mục với notebook
df = pd.read_csv('online_shoppers_purchasing_intention.csv')

print("Thông tin tổng quan về dữ liệu:")
df.info()

print("\n5 dòng dữ liệu đầu tiên:")
print(df.head())


In [ ]:
# --- 3. TRỰC QUAN HÓA DỮ LIỆU CƠ BẢN ---

# 3.1. Phân phối của biến mục tiêu 'Revenue' -> Thấy rõ sự mất cân bằng
plt.figure(figsize=(8, 6))
sns.countplot(x='Revenue', data=df)
plt.title('Phân phối số phiên có mua hàng (True) và không mua hàng (False)')
plt.show()


In [ ]:
# 3.2. Mối quan hệ giữa 'VisitorType' và 'Revenue'
plt.figure(figsize=(10, 6))
sns.countplot(x='VisitorType', hue='Revenue', data=df)
plt.title('Tỷ lệ mua hàng theo loại khách truy cập')
plt.show()


In [ ]:
# 3.3. Phân phối của 'PageValues' đối với hai lớp Revenue
# PageValues: giá trị trung bình của trang mà người dùng đã truy cập trước khi hoàn tất giao dịch.
# Đây thường là một thuộc tính rất quan trọng.
plt.figure(figsize=(12, 7))
sns.histplot(data=df, x='PageValues', hue='Revenue', kde=True, bins=50)
plt.title('Phân phối giá trị trang theo việc mua hàng')
plt.show()


In [ ]:
# 3.4. Heatmap tương quan cho các biến số
plt.figure(figsize=(15, 12))
numerical_cols = df.select_dtypes(include=np.number).columns
sns.heatmap(df[numerical_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Ma trận tương quan giữa các biến số')
plt.show()

In [ ]:
# --- 4. TIỀN XỬ LÝ DỮ LIỆU ---

# Tách features (X) và target (y)
X = df.drop('Revenue', axis=1)
y = df['Revenue'].astype(int) # Chuyển đổi True/False thành 1/0

# Xác định các cột số và hạng mục
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

# Tạo các pipeline xử lý cho từng loại cột
# Pipeline số: Chuẩn hóa dữ liệu
numerical_transformer = StandardScaler()
# Pipeline hạng mục: One-hot encoding
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Sử dụng ColumnTransformer để áp dụng các pipeline trên đúng cột
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Phân chia dữ liệu -> Sử dụng stratify=y là cực kỳ quan trọng do dữ liệu mất cân bằng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")

In [ ]:
# --- 5. HUẤN LUYỆN MÔ HÌNH CƠ BẢN (CHƯA XỬ LÝ MẤT CÂN BẰNG) ---

# Tạo pipeline hoàn chỉnh: Tiền xử lý -> Mô hình
pipeline_rf_basic = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Huấn luyện
pipeline_rf_basic.fit(X_train, y_train)

# Đánh giá
y_pred_basic = pipeline_rf_basic.predict(X_test)
print("--- KẾT QUẢ MÔ HÌNH CƠ BẢN (RANDOM FOREST) ---")
print(classification_report(y_test, y_pred_basic))

In [ ]:
pipeline_rf_smote = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Huấn luyện
pipeline_rf_smote.fit(X_train, y_train)

# Đánh giá
y_pred_smote = pipeline_rf_smote.predict(X_test)
print("\n--- KẾT QUẢ MÔ HÌNH VỚI SMOTE (RANDOM FOREST) ---")
print(classification_report(y_test, y_pred_smote))

In [ ]:
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__class_weight': ['balanced', None] # Thử cả việc dùng trọng số
}

# Sử dụng pipeline đã có SMOTE làm estimator
# Tối ưu hóa theo 'f1' score cho lớp dương (positive class)
grid_search = GridSearchCV(
    estimator=pipeline_rf_smote,
    param_grid=param_grid,
    scoring='f1',
    cv=3, # Sử dụng 3-fold CV để tiết kiệm thời gian, có thể tăng lên 5
    n_jobs=-1,
    verbose=2
)

print("\nBắt đầu quá trình tinh chỉnh siêu tham số...")
grid_search.fit(X_train, y_train)

In [ ]:
print(f"\nTham số tốt nhất tìm được: {grid_search.best_params_}")
print(f"F1-score tốt nhất trên tập validation: {grid_search.best_score_:.4f}")

In [ ]:
best_model = grid_search.best_estimator_
y_pred_final = best_model.predict(X_test)

print("\n--- KẾT QUẢ MÔ HÌNH CUỐI CÙNG SAU KHI TINH CHỈNH ---")
print(classification_report(y_test, y_pred_final))

print("\nMa trận nhầm lẫn cuối cùng:")
cm = confusion_matrix(y_test, y_pred_final)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

final_f1_score = f1_score(y_test, y_pred_final)
print(f"\nF1-SCORE CUỐI CÙNG TRÊN TẬP TEST: {final_f1_score:.4f}")

if final_f1_score >= 0.65:
    print("\nChúc mừng! Mô hình đã đạt hoặc vượt qua giá trị kỳ vọng (F1 >= 0.65).")
else:
    print("\nMô hình chưa đạt giá trị kỳ vọng. Có thể thử mở rộng param_grid hoặc dùng thuật toán khác như XGBoost.")